## BM25

In [1]:
import re
import json
import time
import codecs
import string
import pickle

from tqdm import tqdm
from pathlib import Path
from rank_bm25 import BM25Okapi
from collections import OrderedDict

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [2]:
def clean(text):

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))
    
    text = text.lower().strip()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([w for w in word_tokenize(text) if not w in stop_words])
    text = ' '.join([lemmatizer.lemmatize(w) for w in word_tokenize(text) if not w in stop_words])
    
    return text

In [3]:
Data = json.load(open('../Dumps/corpus.json','r',encoding='utf-8',errors='ignore'))

In [4]:
corpus = ['' for i in range(len(Data))]

for i in tqdm(Data):
    Tokens = word_tokenize(clean(' '.join(Data[i])))
    corpus[int(i)] = Tokens

100%|██████████| 494/494 [05:01<00:00,  1.64it/s]


In [5]:
bm25 = BM25Okapi(corpus)

In [6]:
pickle.dump(corpus,open('../Dumps/bm25_corpus.pkl','wb'))
pickle.dump(bm25,open('../Dumps/bm25.pkl','wb'))

In [4]:
corpus = pickle.load(open('../Dumps/bm25_corpus.pkl','rb'))
bm25 = pickle.load(open('../Dumps/bm25.pkl','rb'))
Index = pickle.load(open('../Dumps/index.pkl','rb'))
company = pickle.load(open('../Dumps/company_index.pkl','rb'))

In [5]:
query = list(clean(input()).split())
Scores = bm25.get_scores(query)

Ranking = sorted( [(Scores[i],i) for i in range(len(Scores))] ,reverse=True)
Ranking[:10]

macbook


[(8.86212612378395, 483),
 (8.331892280675806, 472),
 (8.053576405847107, 461),
 (6.999546701616093, 442),
 (6.9630544279795705, 451),
 (0.0, 493),
 (0.0, 492),
 (0.0, 491),
 (0.0, 490),
 (0.0, 489)]

In [8]:
Best_doc_corpus = Data[str(Ranking[0][1])]
bm25_sen = BM25Okapi([i.split() for i in Best_doc_corpus])
bm25_sen.get_top_n(query,Best_doc_corpus, n=1)

['As of September\xa028, 2019, a portion of the Company’s Japanese yen–denominated notes with a carrying value of $']